In [13]:
import numpy as np
import scipy as sp 
from DCBC.dcbc import compute_DCBC_np 
import DCBC.utilities as ut
import Functional_Fusion.atlas_map as am

In [12]:

dcbc_values = compute_DCBC_np(maxDist=35, binWidth=1, parcellation=np.empty([]),
                    func=None, dist=None, weighting=True)

ValueError: einstein sum subscripts string contains too many subscripts for operand 0

In [51]:
atlas, _ = am.get_atlas('MNISymThalamus1')

#print(np.isnan(atlas.world).sum())  # count of NaNs
#print(atlas.world.min(), atlas.world.max()) 


coords = atlas.world.T

print(coords)

dist = ut.euclidean_distance(coords,coords,decimals=6)

print(dist.shape)



[[-29. -29.   4.]
 [-29. -29.   5.]
 [-29. -29.   6.]
 ...
 [ 29. -18.   6.]
 [ 29. -18.   7.]
 [ 29. -18.   8.]]
(25640, 25640)


In [52]:
print(dist[6])

n_nans = np.isnan(dist).sum()

print("Number of NaNs in dist:", n_nans)

prop_nans = np.isnan(dist).mean()
print(f"Proportion of NaNs: {prop_nans:.6f}")

[      nan       nan       nan ... 62.561969 62.545983 62.529993]
Number of NaNs in dist: 137128484
Proportion of NaNs: 0.208589
